In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectKBest, f_classif
from scipy.stats import ttest_ind
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

## 데이터 수집 및 전처리
### kosdaq

In [ ]:
## kosdqp
## csv 파일로부터 재무제표 데이터를 불러옴
kosdaq1 = pd.read_csv("./코스닥데이터(연결재무제표).csv", encoding = "euc-kr")
kosdaq2 = pd.read_csv("./코스닥데이터(개별재무제표).csv", encoding = "euc-kr")
kosdaq1.columns = ["회사명", "거래소코드", "회계년도", "매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율", "상장폐지일", "소속코드", "상장일"]
kosdaq2.columns = ["회사명", "거래소코드", "회계년도", "매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율", "상장폐지일", "소속코드", "상장일"]
## 결측치를 개별재무제표로 대체함
kosdaq = kosdaq1.fillna(kosdaq2)
## 해당년도 재무제표가 존재하지 않는 기업 해당년도 삭제
# kosdaq.isna().sum()
kosdaq.dropna(subset=["매출액증가율"], inplace = True)
# kosdaq.isna().sum()
# 상장일 이전에 회계년도 데이터 삭제
kosdaq['상장일'] = pd.to_datetime(kosdaq['상장일'])
kosdaq['회계년도'] = pd.to_datetime(kosdaq['회계년도'], format='%Y/%m')
kosdaq = kosdaq.loc[kosdaq['회계년도'] >= kosdaq['상장일']]
## kosdaq 외국계 기업 종목코드 제거
kosdaq = kosdaq[kosdaq['거래소코드'] < 900000]
## 2. 거래소코드 2개인 기업들 셀렉
# grouped = kosdaq.groupby('회사명')['거래소코드'].nunique()
# result = grouped[grouped > 1].index.tolist()
# print(result)
## 거래소코드 2개인 기업 확인하여 삭제
codes = [271400, 60540, 262760, 278380, 131390, 230400, 284610, 176560, 284420, 251960, 216280,25440, 229480]
kosdaq = kosdaq.loc[~(kosdaq['거래소코드'].isin(codes))]
## 증가율 0 인 데이터 평균값으로 대체
kosdaq['자기자본증가율'] = kosdaq['자기자본증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['자기자본증가율'].mean()
kosdaq['자기자본증가율'] = kosdaq['자기자본증가율'].fillna(mean_growth_rate)
kosdaq['순이익증가율'] = kosdaq['순이익증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['순이익증가율'].mean()
kosdaq['순이익증가율'] = kosdaq['순이익증가율'].fillna(mean_growth_rate)
kosdaq['매출액증가율'] = kosdaq['매출액증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['매출액증가율'].mean()
kosdaq['매출액증가율'] = kosdaq['매출액증가율'].fillna(mean_growth_rate)
kosdaq['총자본증가율'] = kosdaq['총자본증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['총자본증가율'].mean()
kosdaq['총자본증가율'] = kosdaq['총자본증가율'].fillna(mean_growth_rate)
kosdaq['유형자산증가율'] = kosdaq['유형자산증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['유형자산증가율'].mean()
kosdaq['유형자산증가율'] = kosdaq['유형자산증가율'].fillna(mean_growth_rate)
kosdaq['비유동자산증가율'] = kosdaq['비유동자산증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['비유동자산증가율'].mean()
kosdaq['비유동자산증가율'] = kosdaq['비유동자산증가율'].fillna(mean_growth_rate)
kosdaq['유동자산증가율'] = kosdaq['유동자산증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['유동자산증가율'].mean()
kosdaq['유동자산증가율'] = kosdaq['유동자산증가율'].fillna(mean_growth_rate)
kosdaq['재고자산증가율'] = kosdaq['재고자산증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['재고자산증가율'].mean()
kosdaq['재고자산증가율'] = kosdaq['재고자산증가율'].fillna(mean_growth_rate)
kosdaq = kosdaq.astype({"거래소코드" : "str"})
kosdaq = kosdaq.astype({"소속코드" : "str"})
kosdaq

In [ ]:
# 윈저라이징으로 양쪽 이상치 처리
kosdaq['매출액증가율']

q5 = np.percentile(kosdaq['매출액증가율'], 5)
q95 = np.percentile(kosdaq['매출액증가율'], 95)

outliers = kosdaq['매출액증가율'][(kosdaq['매출액증가율'] < q5) | (kosdaq['매출액증가율'] > q95)]

print("하위 5%:", q5)
print("상위 5%:", q95)
print("이상치 :", outliers)

In [ ]:
# 윈저라이징 함수 만들기
def windsorize(data, lower_percentile=0.05, upper_percentile=0.95):
    if isinstance(data, pd.Series):
        data = pd.DataFrame(data)
    numeric_df = data.select_dtypes(include=np.number)
    lower_bound = np.percentile(numeric_df, lower_percentile * 100)
    upper_bound = np.percentile(numeric_df, upper_percentile * 100)
    windsorized_data = np.where(numeric_df < lower_bound, lower_bound, numeric_df)
    windsorized_data = np.where(windsorized_data > upper_bound, upper_bound, windsorized_data)
    return windsorized_data

In [ ]:
for i in range(3, 50, 1):
    kosdaq[kosdaq.columns[i]] = windsorize(kosdaq[kosdaq.columns[i]])

In [ ]:
# 전후 비교
kosdaq['매출액증가율']

q5 = np.percentile(kosdaq['매출액증가율'], 5)
q95 = np.percentile(kosdaq['매출액증가율'], 95)

outliers = kosdaq['매출액증가율'][(kosdaq['매출액증가율'] < q5) | (kosdaq['매출액증가율'] > q95)]

print("하위 5%:", q5)
print("상위 5%:", q95)
print("이상치:", outliers)

In [ ]:
# standard 스케일링
numeric_cols = kosdaq.select_dtypes(include=np.number).columns
scaler = StandardScaler()
kosdaq[numeric_cols] = scaler.fit_transform(kosdaq[numeric_cols])

print(kosdaq.head())

In [ ]:
# 체크하는 법
kosdaq[numeric_cols].mean()

In [ ]:
kosdaq[numeric_cols].std()

### kospi

In [ ]:
kospi1 = pd.read_csv("./코스피데이터(연결재무제표).csv", encoding = "euc-kr")
kospi2 = pd.read_csv("./코스피데이터(개별재무제표).csv", encoding = "euc-kr")
kospi1.columns = ["회사명", "거래소코드", "회계년도", "매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율", "상장폐지일", "소속코드", "상장일"]
kospi2.columns = ["회사명", "거래소코드", "회계년도", "매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율", "상장폐지일", "소속코드", "상장일"]
kospi = kospi1.fillna(kospi2)
kospi.dropna(subset=["매출액증가율"], inplace = True)
kospi['상장일'] = pd.to_datetime(kospi['상장일'])
kospi['회계년도'] = pd.to_datetime(kospi['회계년도'], format='%Y/%m')
kospi = kospi.loc[kospi['회계년도'] >= kospi['상장일']]
kospi = kospi[kospi['거래소코드'] < 900000]
kospi['자기자본증가율'] = kospi['자기자본증가율'].replace(0, np.nan)
mean_growth_rate = kospi['자기자본증가율'].mean()
kospi['자기자본증가율'] = kospi['자기자본증가율'].fillna(mean_growth_rate)
kospi['순이익증가율'] = kospi['순이익증가율'].replace(0, np.nan)
mean_growth_rate = kospi['순이익증가율'].mean()
kospi['순이익증가율'] = kospi['순이익증가율'].fillna(mean_growth_rate)
kospi['유동자산증가율'] = kospi['유동자산증가율'].replace(0, np.nan)
mean_growth_rate = kospi['유동자산증가율'].mean()
kospi['유동자산증가율'] = kospi['유동자산증가율'].fillna(mean_growth_rate)
kospi['재고자산증가율'] = kospi['재고자산증가율'].replace(0, np.nan)
mean_growth_rate = kospi['재고자산증가율'].mean()
kospi['재고자산증가율'] = kospi['재고자산증가율'].fillna(mean_growth_rate)
kospi['유형자산증가율'] = kospi['유형자산증가율'].replace(0, np.nan)
mean_growth_rate = kospi['유형자산증가율'].mean()
kospi['유형자산증가율'] = kospi['유형자산증가율'].fillna(mean_growth_rate)
kospi['비유동자산증가율'] = kospi['비유동자산증가율'].replace(0, np.nan)
mean_growth_rate = kospi['비유동자산증가율'].mean()
kospi['비유동자산증가율'] = kospi['비유동자산증가율'].fillna(mean_growth_rate)
kospi['매출액증가율'] = kospi['매출액증가율'].replace(0, np.nan)
mean_growth_rate = kospi['매출액증가율'].mean()
kospi['매출액증가율'] = kospi['매출액증가율'].fillna(mean_growth_rate)
kospi['총자본증가율'] = kospi['총자본증가율'].replace(0, np.nan)
mean_growth_rate = kospi['총자본증가율'].mean()
kospi['총자본증가율'] = kospi['총자본증가율'].fillna(mean_growth_rate)
kospi = kospi.astype({"거래소코드" : "str"})
kospi = kospi.astype({"소속코드" : "str"})
kospi

In [ ]:
for i in range(3, 50, 1):
    kospi[kospi.columns[i]] = windsorize(kospi[kospi.columns[i]])

In [ ]:
# 전후 비교
kospi['매출액증가율']

q5 = np.percentile(kospi['매출액증가율'], 5)
q95 = np.percentile(kospi['매출액증가율'], 95)

outliers = kospi['매출액증가율'][(kospi['매출액증가율'] < q5) | (kospi['매출액증가율'] > q95)]

print("하위 5%:", q5)
print("상위 5%:", q95)
print("이상치:", outliers)

In [ ]:
# standard 스케일링
numeric_cols = kospi.select_dtypes(include=np.number).columns
scaler = StandardScaler()
kospi[numeric_cols] = scaler.fit_transform(kospi[numeric_cols])

print(kospi.head())

In [ ]:
# kospi 상장폐지 기업
kospi_sp = kospi.loc[~(kospi["상장폐지일"].isna())]
kospi_sp["회사명"].unique()

In [ ]:
# kosdaq 상장폐지 기업
kosdaq_sp = kosdaq.loc[~(kosdaq["상장폐지일"].isna())]
kosdaq_sp["회사명"].unique()

#### krx 상장폐지 현황 데이터의 상장폐지사유와 비교하여 부실여부 판단

In [ ]:
# 20180101 ~ 20221231 상장폐지종목 데이터
sp = pd.read_excel("./상장폐지기업.xlsx")
cond = (sp["증권구분"] == "주권") & (sp["주식종류"] == "보통주") & (sp['시장구분'] != 'KONEX')
sp = sp[cond]
sp = sp.drop('번호', axis=1)
sp.to_excel("delist.xlsx")

In [ ]:
sp.loc[sp['종목명'].isin(['동성화학', '부산도시가스', '사조해표', '신한', '엔에스쇼핑',
       '폴루스바이오팜', '한화갤러리아타임월드', '고려개발', '두산건설', '롯데푸드',
       '알보젠코리아', '에스비에스미디어홀딩스', '웅진에너지', '코리아오토글라스',
       '코오롱머티리얼', '한국제지', 'SBS미디어홀딩스', '부산가스'])]

In [ ]:
# krx 데이터에서 kospi 상장폐지 종목 확인
sp.loc[sp['종목명'].isin(['동성화학', '부산도시가스', '사조해표', '신한', '엔에스쇼핑',
       '폴루스바이오팜', '한화갤러리아타임월드', '고려개발', '두산건설', '롯데푸드',
       '알보젠코리아', '에스비에스미디어홀딩스', '웅진에너지', '코리아오토글라스',
       '코오롱머티리얼', '한국제지', 'SBS미디어홀딩스', '부산가스'])]

In [ ]:
# kospi 중 부실, 정상 기업 추출
kospi_bs = kospi.loc[kospi['회사명'].isin(['(주)동성화학', '(주)사조해표', '(주)신한', '(주)폴루스바이오팜', 
                                        '고려개발(주)', '롯데푸드(주)', '에스비에스미디어홀딩스(주)', 
                                        '웅진에너지(주)', '코리아오토글라스(주)', '한국제지(주)'])]
kospi_js = kospi.loc[~kospi['회사명'].isin(['(주)동성화학', '(주)사조해표', '(주)신한', '(주)폴루스바이오팜', 
                                         '고려개발(주)', '롯데푸드(주)', '에스비에스미디어홀딩스(주)', 
                                         '웅진에너지(주)', '코리아오토글라스(주)', '한국제지(주)'])]

In [ ]:
# krx 데이터에서 kosdaq 상장폐지 종목 확인
sp.loc[sp['종목명'].isin(['금빛', '녹십자셀', '뉴로스', '데코앤이', '드림티엔터테인먼트', '레드로버',
       '맘스터치앤컴퍼니', '맥스로텍', '모다', '미래SCI', '미래테크놀로지',
       '바이오빌', '세영디앤씨', '소리바다', '썬테크놀로지스', '씨엠에스에듀',
       '아이엠텍', '에스마크', '에스에이티', '에스에프씨', '에스제이케이',
       '에이치디', '에이팸', '에프티이앤이', '엘엑스세미콘', '엘피케이로보틱스',
       '엠씨넥스', '연이비앤티', '원익피앤이', '이매진아시아', '이십일스토어',
       '이엠네트웍스', '인터파크', '제낙스', '제이콘텐트리', '제이테크놀로지',
       '지에스홈쇼핑', '지유온', '카이노스메드', '켐온', '코너스톤네트웍스',
       '코리아센터', '쿠첸', '티씨엠생명과학', '티에스트릴리온', '파인넥스',
       '파티게임즈', '팍스넷', '포스코케미칼', '포인트엔지니어링', '폭스소프트',
       '퓨전', '피앤텔', '한국비엔씨', '한국아트라스비엑스', '한컴지엠디', '한프',
       '행남사', '화신테크', '화진', '넥슨지티', '다이노나', '디에스티',
       '럭슬', '미래오토스', '에스에이치엔엘', '에스지에이클라우드서비스', '에스케이 머티리얼즈',
       '에스케이씨솔믹스', '에이아이비트', '원텍', '이엘케이', '자안바이오',
       '지스마트글로벌', '참존글로벌', '케이맥', '피아이첨단소재', '한일네트웍스',
       '현대정보기술', '현진소재'])]

In [ ]:
# kosdaq 중 부실, 정상 기업 추출
kosdaq_bs = kosdaq.loc[kosdaq['회사명'].isin(['(주)썬테크놀로지스', '참존글로벌(주)', '(주)레드로버', '(주)뉴로스', '(주)연이비앤티', 
'(주)소리바다', '에스에이치엔엘(주)', '(주)세영디앤씨', '현진소재(주)', '(주)한프',
'지스마트글로벌(주)', '자안바이오(주)', '(주)이엠네트웍스', '(주)코너스톤네트웍스', 
'에이아이비트(주)', '(주)지유온', '(주)맥스로텍', '(주)에스제이케이', '디에스티(주)', 
'(주)팍스넷', '럭슬(주)', '(주)제낙스', '(주)아이엠텍', '(주)퓨전', '(주)행남사',
'(주)에이팸', '(주)에이치디', '(주)미래SCI', '(주)이매진아시아', '(주)파티게임즈', 
'(주)바이오빌', '(주)모다', '(주)화진', '(주)에스에프씨', '이엘케이(주)', '(주)에스마크',
'(주)피앤텔', '(주)파인넥스', '(주)제이테크놀로지', '(주)데코앤이', '(주)에프티이앤이',
'(주)이십일스토어', '(주)화신테크', '미래오토스', '에스지에이클라우드서비스(주)'])]

kosdaq_js = kosdaq.loc[~kosdaq['회사명'].isin(['(주)썬테크놀로지스', '참존글로벌(주)', '(주)레드로버', '(주)뉴로스', '(주)연이비앤티', 
'(주)소리바다', '에스에이치엔엘(주)', '(주)세영디앤씨', '현진소재(주)', '(주)한프',
'지스마트글로벌(주)', '자안바이오(주)', '(주)이엠네트웍스', '(주)코너스톤네트웍스', 
'에이아이비트(주)', '(주)지유온', '(주)맥스로텍', '(주)에스제이케이', '디에스티(주)', 
'(주)팍스넷', '럭슬(주)', '(주)제낙스', '(주)아이엠텍', '(주)퓨전', '(주)행남사',
'(주)에이팸', '(주)에이치디', '(주)미래SCI', '(주)이매진아시아', '(주)파티게임즈', 
'(주)바이오빌', '(주)모다', '(주)화진', '(주)에스에프씨', '이엘케이(주)', '(주)에스마크',
'(주)피앤텔', '(주)파인넥스', '(주)제이테크놀로지', '(주)데코앤이', '(주)에프티이앤이',
'(주)이십일스토어', '(주)화신테크', '미래오토스', '에스지에이클라우드서비스(주)'])]

In [ ]:
# kospi 데이터에 "부실여부" 파생변수 삽입
kospibs = '(주)동성화학', '(주)사조해표', '(주)신한', '(주)폴루스바이오팜', '고려개발(주)', '롯데푸드(주)', '에스비에스미디어홀딩스(주)', '웅진에너지(주)', '코리아오토글라스(주)', '한국제지(주)'
kospi["부실여부"] = kospi['회사명'].apply(lambda x: 1 if x in kospibs else 0)
kospi.head()

In [ ]:
# kosdaq 데이터에 "부실여부" 파생변수 삽입
kosdaqbs = '(주)썬테크놀로지스', '참존글로벌(주)', '(주)레드로버', '(주)뉴로스', '(주)연이비앤티', 
'(주)소리바다', '에스에이치엔엘(주)', '(주)세영디앤씨', '현진소재(주)', '(주)한프',
'지스마트글로벌(주)', '자안바이오(주)', '(주)이엠네트웍스', '(주)코너스톤네트웍스', 
'에이아이비트(주)', '(주)지유온', '(주)맥스로텍', '(주)에스제이케이', '디에스티(주)', 
'(주)팍스넷', '럭슬(주)', '(주)제낙스', '(주)아이엠텍', '(주)퓨전', '(주)행남사',
'(주)에이팸', '(주)에이치디', '(주)미래SCI', '(주)이매진아시아', '(주)파티게임즈', 
'(주)바이오빌', '(주)모다', '(주)화진', '(주)에스에프씨', '이엘케이(주)', '(주)에스마크',
'(주)피앤텔', '(주)파인넥스', '(주)제이테크놀로지', '(주)데코앤이', '(주)에프티이앤이',
'(주)이십일스토어', '(주)화신테크', '미래오토스', '에스지에이클라우드서비스(주)'
kosdaq["부실여부"] = kosdaq['회사명'].apply(lambda x: 1 if x in kosdaqbs else 0)
kosdaq.head()

In [ ]:
# kospi, kosdaq 데이터 병합
data = pd.concat([kospi, kosdaq], axis = 0)

In [ ]:
data

## 피쳐선정

In [ ]:
# lasso

data_feature = data[["매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", 
          "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", 
          "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", 
          "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", 
          "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", 
          "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", 
          "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", 
          "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율"]]

feature = data_feature
target = data['부실여부']

lasso = SelectFromModel(LogisticRegression(C=0.2, penalty='l1',solver='liblinear'))
lasso.fit(feature, target)
lasso_support = lasso.get_support()
lr_feature = feature.loc[:,lasso_support].columns.tolist()
lr_feature

In [ ]:
# selectKBest

data_feature = data[["매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", 
          "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", 
          "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", 
          "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", 
          "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", 
          "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", 
          "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", 
          "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율"]]

# 독립변수와 종속변수를 분리합니다.
X = data_feature
y = data['부실여부']

# f 검정 방법을 사용해 피처를 선택합니다.'k=선정개수'
selector = SelectKBest(score_func=f_classif, k=20)
X_new = selector.fit_transform(X, y)

# 선택된 피처를 출력합니다.
print(X.columns[selector.get_support()])

## 모델링 및 성능평가

In [ ]:
# 랜덤포레스트

# 학습용 데이터와 테스트용 데이터 분할
train = data[data['회계년도'] < '2021-01-01']
test = data[data['회계년도'] >= '2021-01-01']

# 학습용 데이터 전처리
X_train = train[["현금비율", "순운전자본비율", '매출액순이익률', "총자본사업이익률", "부가가치율"]]
y_train = train['부실여부']

# 테스트용 데이터 전처리
X_test = test[["현금비율", "순운전자본비율", '매출액순이익률', "총자본사업이익률", "부가가치율"]]
y_test = test['부실여부']

# 랜덤 포레스트 모델 학습
rf = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=1)
rf.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = rf.predict(X_test)

# 평가 지표 출력
mse = mean_squared_error(y_test, y_pred)
print('MSE:', mse)

# 정확도 계산 및 출력
accuracy = accuracy_score(y_test, y_pred.round())
print('Accuracy:', accuracy)

In [ ]:
# 로지스틱

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 모델 선택 및 훈련
model = LogisticRegression()
model.fit(X_train, y_train)

# 모델 평가
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

## 지수화
### 코스피200 대형주 50% 그 외 50% 랜덤추출하여 60종목 편입

In [155]:
df1 = pd.read_excel("./2020코스피200_1분기.xlsx")
df2 = pd.read_excel("./2020코스피200_반기.xlsx")
Scale = pd.read_csv("./기업규모코드.csv", encoding = "euc-kr")

In [ ]:
# df1["회사명"].unique()

In [ ]:
# df2["회사명"].unique()

In [156]:
common_companies = pd.Series(list(set(df1['회사명']) & set(df2['회사명']) & set(Scale['회사명'])))

df1 = df1[df1['회사명'].isin(common_companies)]
df2 = df2[df2['회사명'].isin(common_companies)]
Scale = Scale[Scale['회사명'].isin(common_companies)]

In [157]:
df1 = df1.reset_index(drop = True)
df2 = df2.reset_index(drop = True)

In [ ]:
df2

In [158]:
df1['기업규모코드'] = Scale['기업규모코드']
df2['기업규모코드'] = Scale['기업규모코드']

In [ ]:
df1

In [ ]:
df2

In [159]:
df1.columns = ['회사명', '거래소코드', '회계년도', '자산', '현금및현금성자산', '부채', '유동자산', '유동부채', '원재료', '선급공사비,외주비', '매출액', '법인세비용차감전순이익', '법인세비용', '당기순이익', '기업규모코드']
df2.columns = ['회사명', '거래소코드', '회계년도', '자산', '현금및현금성자산', '부채', '유동자산', '유동부채', '원재료', '선급공사비,외주비', '매출액', '법인세비용차감전순이익', '법인세비용', '당기순이익', '기업규모코드']

In [ ]:
df_1 = df1[['자산', '현금및현금성자산', '부채', '유동자산', '유동부채', '원재료', '선급공사비,외주비', '매출액', '법인세비용차감전순이익', '법인세비용', '당기순이익']]
df_2 = df2[['자산', '현금및현금성자산', '부채', '유동자산', '유동부채', '원재료', '선급공사비,외주비', '매출액', '법인세비용차감전순이익', '법인세비용', '당기순이익']]

In [160]:
df_2 = df_2 - df_1

In [ ]:
df_2

In [161]:
d1 = pd.concat([df1[['회사명', '거래소코드', '회계년도', '기업규모코드']], df_1], axis=1)
d2 = pd.concat([df2[['회사명', '거래소코드', '회계년도', '기업규모코드']], df_2], axis=1)

In [163]:
d2

,회사명,거래소코드,회계년도,기업규모코드,자산,현금및현금성자산,부채,유동자산,유동부채,원재료,"선급공사비,외주비",매출액,법인세비용차감전순이익,법인세비용,당기순이익
0,(주)BNK금융지주,138930,2020/06,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(주)DB하이텍,990,2020/06,10.0,-2.323554e+09,-1.629522e+08,-1.003938e+09,-1.026109e+09,-4.535347e+08,-9.386985e+07,0.0,-2.099550e+08,-63621518.0,-3844475.0,-59777042.0
2,(주)JB금융지주,175330,2020/06,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,(주)KB금융지주,105560,2020/06,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,(주)강원랜드,35250,2020/06,30.0,-9.192935e+09,-1.153198e+08,-2.393538e+09,-4.585121e+09,-2.265065e+09,-4.715795e+06,0.0,-4.373802e+08,368396603.0,101723534.0,266673069.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,현대자동차(주),5380,2020/06,10.0,-3.947480e+11,-2.023774e+10,-2.430163e+11,-8.965547e+10,-1.084062e+11,-3.415237e+09,0.0,-2.877990e+10,-852270000.0,-124176000.0,-728094000.0
183,현대제철(주),4020,2020/06,10.0,-6.793976e+10,-2.632866e+09,-3.464125e+10,-1.959647e+10,-1.384758e+10,-2.330962e+09,0.0,-5.222725e+09,302274798.0,84331682.0,217943116.0
184,현대해상화재보험(주),1450,2020/06,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185,효성첨단소재(주),298050,2020/06,10.0,-5.722741e+09,-2.360598e+08,-4.837872e+09,-2.442863e+09,-3.577343e+09,-2.144196e+08,0.0,-9.990877e+08,-78268920.0,-21671629.0,-56597294.0


In [164]:
d1['현금비율'] = (d1['현금및현금성자산'] / d1['부채']) * 100
d1['순운전자본비율'] = ((d1['유동자산'] - d1['유동부채']) / d1['매출액']) * 100
d1['매출액순이익률'] = ((d1['법인세비용차감전순이익'] - d1['법인세비용']) / d1['매출액']) * 100
d1['총자본사업이익률'] = (d1['당기순이익'] / (d1['자산'] - d1['부채'])) * 100
d1['부가가치율'] = ((d1['매출액'] - d1['원재료'] - d1['선급공사비,외주비']) / d1['매출액']) * 100

In [165]:
d2['현금비율'] = (d2['현금및현금성자산'] / d2['부채']) * 100
d2['순운전자본비율'] = ((d2['유동자산'] - d2['유동부채']) / d2['매출액']) * 100
d2['매출액순이익률'] = ((d2['법인세비용차감전순이익'] - d2['법인세비용']) / d2['매출액']) * 100
d2['총자본사업이익률'] = (d2['당기순이익'] / (d2['자산'] - d2['부채'])) * 100
d2['부가가치율'] = ((d2['매출액'] - d2['원재료'] - d2['선급공사비,외주비']) / d2['매출액']) * 100

In [166]:
d1 = d1[['회사명', '거래소코드', '회계년도', '기업규모코드', "현금비율", "순운전자본비율", '매출액순이익률', "총자본사업이익률", "부가가치율"]]
d2 = d2[['회사명', '거래소코드', '회계년도', '기업규모코드', "현금비율", "순운전자본비율", '매출액순이익률', "총자본사업이익률", "부가가치율"]]

In [ ]:
# total_d1 = d1.dropna()

# # 기업규모코드가 10인 데이터 추출
# scale_10_ = total_d1.loc[total_d1['기업규모코드'] == 10]

# # 기업규모코드가 10이 아닌 데이터 추출
# scale_N10_ = total_d1.loc[total_d1['기업규모코드'] != 10]

# # scale_10에서 랜덤하게 30개 추출
# scale_10 = scale_10_.sample(n=30, random_state=42)

# # scale_N10에서 랜덤하게 30개 추출
# scale_N10 = scale_N10_.sample(n=30, random_state=42)

# # 추출한 기업들을 하나로 합치기
# selected_companies = pd.concat([scale_10, scale_N10])

In [169]:
d1 = d1.dropna().reset_index(drop=True)
d2 = d2.dropna().reset_index(drop=True)

In [170]:
d1

,회사명,거래소코드,회계년도,기업규모코드,현금비율,순운전자본비율,매출액순이익률,총자본사업이익률,부가가치율
0,(주)DB하이텍,990,2020/03,10.0,13.955806,105.403498,24.386218,8.039872,79.085448
1,(주)강원랜드,35250,2020/03,30.0,8.319922,453.677755,-66.149280,-4.624698,99.010158
2,(주)녹십자,6280,2020/03,30.0,6.423597,172.856576,-1.274386,-0.347217,76.353100
3,(주)녹십자홀딩스,5250,2020/03,30.0,9.918991,110.941967,-2.804519,-0.693253,75.161082
4,(주)농심,4370,2020/03,30.0,48.369342,74.944711,7.106862,2.450882,93.059396
...,...,...,...,...,...,...,...,...,...
149,현대위아(주),11210,2020/03,30.0,7.967116,108.523466,4.642614,2.416484,83.475294
150,현대자동차(주),5380,2020/03,10.0,8.439213,-31.768256,2.182836,0.727916,92.759420
151,현대제철(주),4020,2020/03,10.0,8.076573,66.827805,-2.473020,-0.692453,74.428360
152,효성첨단소재(주),298050,2020/03,10.0,4.401035,-98.696916,1.208208,1.990544,82.616668


In [171]:
d2

,회사명,거래소코드,회계년도,기업규모코드,현금비율,순운전자본비율,매출액순이익률,총자본사업이익률,부가가치율
0,(주)DB하이텍,990,2020/06,10.0,16.231297,272.712968,28.471356,4.529883,55.290498
1,(주)강원랜드,35250,2020/06,30.0,4.817963,530.443760,-60.970546,-3.922011,98.921809
2,(주)녹십자,6280,2020/06,30.0,2.061693,460.947810,-8.387896,-0.950334,41.508127
3,(주)녹십자홀딩스,5250,2020/06,30.0,7.327551,332.424339,-16.472575,-1.717455,39.681912
4,(주)농심,4370,2020/06,30.0,48.781326,143.610101,8.672249,1.547965,88.409541
...,...,...,...,...,...,...,...,...,...
149,현대위아(주),11210,2020/06,30.0,6.510718,172.783979,9.673555,3.155356,73.826273
150,현대자동차(주),5380,2020/06,10.0,8.327728,-65.152025,2.529869,0.479856,88.133258
151,현대제철(주),4020,2020/06,10.0,7.600376,110.074587,-4.172977,-0.654513,55.368844
152,효성첨단소재(주),298050,2020/06,10.0,4.879414,-113.551528,5.664897,6.396123,78.538460


In [ ]:
# d1['현금비율'].sort_values(ascending=False).reset_index(drop=True).loc[[14, 30, 45, 60, 76, 91, 107, 122, 138, 153]]

In [172]:
# 데이터프레임의 열 이름을 리스트로 만듭니다.
columns = d1.columns.tolist()[4:]

# 각 열에서 상위 10개의 데이터를 추출하는 for문을 작성합니다.
result = []
for col in columns:
    data = d1[col].sort_values(ascending=False).reset_index(drop=True).loc[[14, 30, 45, 60, 76, 91, 107, 122, 138, 153]]
    result.append(data)

# 추출한 결과를 데이터프레임으로 합쳐줍니다.
df_top10 = pd.concat(result, axis=1)

# 데이터프레임을 출력합니다.
print(df_top10)

          현금비율      순운전자본비율      매출액순이익률    총자본사업이익률       부가가치율
14   42.669970   192.873323    13.207272    5.068999  100.000000
30   31.176770   140.669798     9.015971    3.454742   99.702030
45   21.404190   101.134585     7.046674    2.388860   97.680311
60   17.256386    79.626484     5.512844    1.971831   92.929144
76   14.434573    53.978369     3.686783    1.407884   90.544267
91   11.581141    31.368602     2.305742    1.017760   86.705140
107   8.319922    13.481653     0.451564    0.374696   83.595973
122   7.001685    -9.435525    -1.243232   -0.463690   79.727531
138   4.108714   -51.952946    -7.794690   -4.272414   73.089482
153   0.962602 -2871.530690 -1695.489900 -324.956135 -117.818673
